In [ ]:
import pandas as pd
from json import load
from metrics import get_total_metrics
from graphics import get_3d_plot, get_2d_plot

In [ ]:
# Load target sequences and drugs from respective JSON files
targets = load(open('../data/KIBA/target_seq.txt'))
drugs = load(open('../data/KIBA/SMILES.txt'))

In [ ]:
# Change the location of the file to process
unpivoted = pd.read_csv('../analysis/morgan_cnn_kiba_predictions.csv')

unpivoted.columns = ['drug', 'target', 'affinity', 'predicted']
unpivoted['SMILES'] = unpivoted.drug.apply(drugs.get)
unpivoted

In [ ]:
results = get_total_metrics(unpivoted, threshold_affinity=[1, 2, 3, 4], threshold_similarity=[0.1, 0.3, 0.5, 0.7, 0.9])
# takes 18 mins

In [ ]:
# backup
bu = results.copy()

In [ ]:
results.to_csv('../analysis/morgan_cnn_kiba_performance_results.csv')
results

In [ ]:
df = pd.read_csv('../analysis/morgan_cnn_kiba_performance.csv', index_col=0).T

In [ ]:
df

In [ ]:
df.insert(0, 'threshold_similarity', 0)
df.insert(0, 'threshold_affinity', 0)
df2 = pd.concat([df, results], ignore_index=True)
df2.to_csv('../analysis/morgan_cnn_kiba_performance.csv')

In [ ]:
df2

In [ ]:
get_3d_plot(df2[1:], 'rmse_micro', general_performance=df2.loc[0, 'rmse_micro'],
            model_name='morgan_cnn_kiba')
get_3d_plot(df2[1:], 'rmse_macro', general_performance=df2.loc[0, 'rmse_macro'],
            model_name='morgan_cnn_kiba')

In [ ]:
df2_sub = df2[(df2['threshold_similarity'] == 0.0) | (df2['threshold_similarity'] == 0.9)]
df2_sub

In [ ]:
get_2d_plot(df2_sub, 'rmse_micro', 'threshold_affinity')
get_2d_plot(df2_sub, 'rmse_macro', 'threshold_affinity')

In [ ]:
df2_sub2 = df2[(df2['threshold_affinity'] == 0) | (df2['threshold_affinity'] == 4)]
df2_sub2

In [ ]:
get_2d_plot(df2_sub2, 'rmse_micro', 'threshold_similarity')
get_2d_plot(df2_sub2, 'rmse_macro', 'threshold_similarity')